In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
test_df = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')
display(df.head())
display(test_df.head())

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


In [3]:
df = df.drop('essay_id', axis=1)
test_df = test_df.drop('essay_id', axis=1)

In [4]:
le = LabelEncoder()
df['score'] = le.fit_transform(df['score'])

In [5]:
train_split, val_split = train_test_split(df, test_size=0.2, random_state=52, stratify=df['score'])

In [6]:
stopwords = nltk.corpus.stopwords.words("english")

# Init tf-idf
vect_word = TfidfVectorizer(
    max_features=100,
    lowercase=True,
    analyzer="word",
    stop_words=stopwords,
    ngram_range=(1, 3),
    dtype=np.float32
)

# Train tf-idf
X_train = vect_word.fit_transform(train_split["full_text"])
X_val = vect_word.transform(val_split["full_text"])
X_test = vect_word.transform(test_df["full_text"])

y_train = train_split["score"]
y_val = val_split["score"]

In [7]:
# Init logreg model
logreg = LogisticRegression(C=2, random_state=33)
logreg.fit(X_train, y_train)
preds_val = logreg.predict(X_val)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(y_val, preds_val, weights='quadratic')

0.4345167974515829

In [9]:
submission = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv')
submission.head()

,essay_id,score
0,000d118,3
1,000fe60,3
2,001ab80,4


In [10]:
preds_test = logreg.predict(X_test)
submission["score"] = preds_test
submission["score"] = le.inverse_transform(submission["score"])
submission.head()

,essay_id,score
0,000d118,2
1,000fe60,3
2,001ab80,4


In [11]:
submission.to_csv("submission.csv", index=False)